In [0]:
#!pip3 install unidecode

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from unidecode import unidecode

In [0]:
# Define uma variável chamada 'path' que contém o caminho para uma pasta específica vinda da rawzone
# A pasta é localizada dentro de 'PASTA_INTERNA_DA_EXTRACAO' e se chama 'Central de Projetos'
path = 'PASTA_INTERNA_DA_EXTRACAO/Central de Projetos'

In [0]:
# %run é uma "magic command" que executa um script no ambiente atual
# Chamada do caderno "handler" e suas funções
%run ../../handler

In [0]:
# %run é uma "magic command" que executa um script no ambiente atual
# Chamada do caderno sharepointAPI_CP
%run ../../sharepointAPI_CP

In [0]:
# Conexão e leitura do arquivo "Ações" dentro de: yourdatalake\rawzone\DIRETORIO_DA_EXTRACAO\EXTRACAO\PASTA_INTERNA_DA_EXTRACAO/Central de Projetos
acoes = Handler('Ações', path)
acoes.read_from('rawzone')
acoes_df = acoes.df

In [0]:
# Descarte de colunas que não serão utilizadas
acoes_df = acoes_df.drop(\
'__metadata',\
'ID',\
'ContentTypeId',\
'OData__x005f_HasCopyDestinations',\
'OData__x005f_CopySource',\
'owshiddenversion',\
'WorkflowVersion',\
'OData__x005f_UIVersion',\
'OData__x005f_UIVersionString',\
'Attachments',\
'OData__x005f_ModerationStatus',\
'OData__x005f_ModerationComments',\
'InstanceID',\
'Order',\
'GUID',\
'WorkflowInstanceID',\
'FileRef',\
'FileDirRef',\
'Last_x005f_x0020_x005f_Modified',\
'Created_x005f_x0020_x005f_Date',\
'FSObjType',\
'SortBehavior',\
'UniqueId',\
'SyncClientId',\
'ProgId',\
'ScopeId',\
'File_x005f_x0020_x005f_Type',\
'MetaInfo',\
'OData__x005f_Level',\
'OData__x005f_IsCurrentVersion',\
'ItemChildCount',\
'FolderChildCount',\
'Restricted',\
'OriginatorId',\
'NoExecute',\
'ContentVersion',\
'OData__x005f_ComplianceFlags',\
'OData__x005f_ComplianceTag',\
'OData__x005f_ComplianceTagWrittenTime',\
'OData__x005f_ComplianceTagUserId',\
'AccessPolicy',\
'OData__x005f_VirusStatus',\
'OData__x005f_VirusVendorID',\
'OData__x005f_VirusInfo',\
'AppAuthor',\
'AppEditor',\
'SMTotalSize',\
'SMLastModifiedDate',\
'SMTotalFileStreamSize',\
'SMTotalFileCount',\
'ComplianceAssetId',\
'OData__x005f_CommentFlags',\
'OData__x005f_CommentCount',\
'FileLeafRef',\
'Created',\
'Author',\
'TipoAssociacao'
)

In [0]:
# Renomeando as colunas para novos nomes que serão utilizados em seu projeto
nome_antigo = ['Projeto',\
               'Title',\
               'Risco',\
               'Problema',\
               'Modified'
            ]
nome_novo = ['NomedoProjeto',\
             'NomedaAcao',\
             'NomedoRisco',\
             'NomedoProblema',\
             'Modificado'
            ]

acoes_df = rename_columns(acoes_df, nome_antigo, nome_novo)

In [0]:
# Cast date de colunas de data e timestamp
acoes_df = cast_to_date(acoes_df, 'Modificado', 'dd/MM/yyyy HH:mm')
acoes_df = cast_to_date(acoes_df, 'DataLimite', 'dd/MM/yyyy')

In [0]:
# Obtem os nomes das colunas do DataFrame e renomeia removendo os caracteres especiais com unidecode
column_names = acoes_df.columns

for name in column_names:
  acoes_df = acoes_df.withColumnRenamed(name, unidecode(name))

In [0]:
# Escreve a extração em um novo arquivo "Ações" na silverzone no yourdatalake\silverzone\Central_Projetos
acoes = Handler('Ações', path)
acoes.df = acoes_df
acoes.write_on('silverzone')